# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *


In [ ]:
#| export
#| hide
import httpx
import os
import io
from pathlib import Path
import tarfile

In [ ]:
#| hide
capsrv= os.environ.get("CAPROVER_SERVER") 
cappass=os.environ.get("CAPROVER_PASSWORD") 

In [ ]:
#| export
def caplogin(
    server_url:str,  # CapRover server URL (e.g., https://captain.domain.com/)
    server_password:str  # CapRover dashboard password
) -> str:  # Authentication token
    "Login to CapRover and return authentication token"
    response = httpx.post(f"{server_url}api/v2/login", json={"password": server_password})
    if response.status_code != 200: raise Exception(f'Error with server URL: {response}')
    if response.json()['status'] !=100: raise Exception(f'Login error: {response.json()}')
    token = response.json()['data']['token']
    return token

In [ ]:
token = caplogin(capsrv, cappass)
token[:2]

'ey'

In [ ]:
#| export
def create_app(
    server_url:str,  # CapRover server URL
    token:str,  # Authentication token from caplogin
    app_name:str,  # Name of the app to create
    persistent:bool=False  # Whether app needs persistent storage
) -> dict:  # API response
    "Register a new app on CapRover"
    headers = {"x-captain-auth": token}
    body = {"appName": app_name, "hasPersistentData": persistent}
    response = httpx.post(f"{server_url}api/v2/user/apps/appDefinitions/register", json=body, headers=headers)
    if response.status_code != 200: raise Exception(f'HTTP Error: {response}')
    if response.json()['status'] != 100: raise Exception(f'API Error: {response.json()}')
    return response.json()

def update_app(
    server_url:str,  # CapRover server URL
    token:str,  # Authentication token from caplogin
    app_name:str,  # Name of the app to update
    volumes:dict=None,  # Dict of {volumeName: containerPath} for persistent storage
    env_vars:dict=None,  # Dict of {name: value} for environment variables
    instance_count:int=1,  # Number of container instances
    force_ssl:bool=None  # Force HTTPS redirect
) -> dict:  # API response
    "Update app configuration (volumes, env vars, SSL settings)"
    headers = {"x-captain-auth": token}
    body = {"appName": app_name, "instanceCount": instance_count}
    if volumes: body["volumes"] = [{"volumeName": k, "containerPath": v} for k,v in volumes.items()]
    if env_vars: body["envVars"] = [{"key": k, "value": v} for k,v in env_vars.items()]
    if force_ssl is not None: body["forceSsl"] = force_ssl
    response = httpx.post(f"{server_url}api/v2/user/apps/appDefinitions/update", json=body, headers=headers)
    if response.status_code != 200: raise Exception(f'HTTP Error: {response}')
    if response.json()['status'] != 100: raise Exception(f'API Error: {response.json()}')
    return response.json()

def enable_ssl(
    server_url:str,  # CapRover server URL
    token:str,  # Authentication token from caplogin
    app_name:str,  # Name of the app
    timeout:int=120  # Request timeout in seconds (SSL generation can be slow)
) -> dict:  # API response
    "Enable HTTPS/SSL for an app's base domain"
    headers = {"x-captain-auth": token}
    body = {"appName": app_name}
    response = httpx.post(f"{server_url}api/v2/user/apps/appDefinitions/enablebasedomainssl", json=body, headers=headers, timeout=timeout)
    if response.status_code != 200: raise Exception(f'HTTP Error: {response}')
    if response.json()['status'] != 100: raise Exception(f'API Error: {response.json()}')
    return response.json()

def setup_app(
    server_url:str,  # CapRover server URL
    token:str,  # Authentication token from caplogin
    app_name:str,  # Name of the app to create and configure
    volumes:dict=None,  # Dict of {volumeName: containerPath} for persistent storage
    env_vars:dict=None,  # Dict of {name: value} for environment variables
    instance_count:int=1,  # Number of container instances
    force_ssl:bool=True  # Force HTTPS redirect
) -> dict:  # API response
    "Create a new app with SSL enabled and configured settings (combines create_app, enable_ssl, update_app)"
    persistent = volumes is not None and len(volumes) > 0
    create_app(server_url, token, app_name, persistent=persistent)
    enable_ssl(server_url, token, app_name)
    return update_app(server_url, token, app_name, volumes=volumes, env_vars=env_vars, instance_count=instance_count, force_ssl=force_ssl)


In [ ]:
app_name = "mynewapp"
volumes = {"testapi-data": "/app/data"}
env_vars = {"MY_SECRET": "supersecret123"}

setup_app(capsrv, token, app_name, volumes=volumes, env_vars=env_vars)

{'status': 100, 'description': 'Updated App Definition Saved', 'data': {}}

In [ ]:
#| export
def upload_app(
    server_url:str,  # CapRover server URL
    token:str,  # Authentication token from caplogin
    app_name:str,  # Name of the app to deploy to
    tar_buffer:io.BytesIO  # In-memory tar.gz file containing deployment files
) -> dict:  # API response
    "Upload and deploy a tar.gz archive to an existing app"
    headers = {"x-captain-auth": token}
    tar_buffer.seek(0)
    files = {"sourceFile": ("deploy.tar.gz", tar_buffer, "application/gzip")}
    response = httpx.post(f"{server_url}api/v2/user/apps/appData/{app_name}", headers=headers, files=files, timeout=120)
    if response.status_code != 200: raise Exception(f'HTTP Error: {response}')
    if response.json()['status'] != 100: raise Exception(f'API Error: {response.json()}')
    return response.json()

def deploy(
    server_url:str,  # CapRover server URL
    token:str,  # Authentication token from caplogin
    app_name:str,  # Name of the app to deploy to
    script_paths:str|list,  # Path(s) to Python script(s). First script is the entry point
    requirements:list=None  # List of pip requirements. Defaults to ["python-fasthtml"]
) -> dict:  # API response
    "Build and deploy a FastHTML app from Python script(s)"
    if isinstance(script_paths, str): script_paths = [script_paths]
    if requirements is None: requirements = ["python-fasthtml"]
    main_script = Path(script_paths[0]).name
    captain_def = '{"schemaVersion": 2, "dockerfilePath": "./Dockerfile"}'
    dockerfile = f"FROM python:3.12-slim\nWORKDIR /app\nCOPY requirements.txt .\nRUN pip install --no-cache-dir -r requirements.txt\nCOPY . .\nENV PORT=80\nEXPOSE 80\nCMD [\"python\", \"{main_script}\"]"
    reqs = "\n".join(requirements)
    
    files = [("captain-definition", captain_def), ("Dockerfile", dockerfile), ("requirements.txt", reqs)]
    for p in script_paths: files.append((Path(p).name, Path(p).read_text()))
    
    tar_buffer = io.BytesIO()
    with tarfile.open(fileobj=tar_buffer, mode='w:gz') as tar:
        for name, content in files:
            data = content.encode('utf-8')
            info = tarfile.TarInfo(name=name)
            info.size = len(data)
            tar.addfile(info, io.BytesIO(data))
    tar_buffer.seek(0)
    return upload_app(server_url, token, app_name, tar_buffer)

In [ ]:
example_app = """from fasthtml.common import *

app = FastHTML()

@app.route("/")
def home(): return Div(H1("Hello World!"), P("Deployed with CapRover API"))

serve()
"""

Path("example_app.py").write_text(example_app)

deploy(capsrv, token, app_name, "example_app.py", requirements=["python-fasthtml"])

{'status': 100, 'description': 'Deploy is done', 'data': {}}

In [ ]:
#| export
def get_app_logs(
    server_url:str,  # CapRover server URL
    token:str,  # Authentication token from caplogin
    app_name:str,  # Name of the app
    encoding:str='ascii'  # Log encoding
) -> str:  # Runtime logs
    "Get runtime logs from a running app"
    headers = {"x-captain-auth": token}
    response = httpx.get(f"{server_url}api/v2/user/apps/appData/{app_name}/logs", headers=headers, params={"encoding": encoding}, timeout=30)
    if response.status_code != 200: raise Exception(f'HTTP Error: {response}')
    if response.json()['status'] != 100: raise Exception(f'API Error: {response.json()}')
    return response.json()['data']['logs']

def get_app_deployment_logs(
    server_url:str,  # CapRover server URL
    token:str,  # Authentication token from caplogin
    app_name:str  # Name of the app
) -> str:  # Build/deployment logs
    "Get build and deployment logs from an app"
    headers = {"x-captain-auth": token}
    response = httpx.get(f"{server_url}api/v2/user/apps/appData/{app_name}", headers=headers, timeout=30)
    if response.status_code != 200: raise Exception(f'HTTP Error: {response}')
    if response.json()['status'] != 100: raise Exception(f'API Error: {response.json()}')
    return '\n'.join(response.json()['data']['logs']['lines'])

In [ ]:
print(get_app_logs(capsrv, token, app_name))

      _2025-11-28T15:28:57.924022039Z INFO:     Will watch for changes in these directories: ['/app']
      e2025-11-28T15:28:57.924112883Z INFO:     Uvicorn running on http://0.0.0.0:80 (Press CTRL+C to quit)
      W2025-11-28T15:28:57.924220463Z INFO:     Started reloader process [1] using WatchFiles
      92025-11-28T15:28:57.930583787Z Link: http://localhost:80
      D2025-11-28T15:28:58.335346643Z INFO:     Started server process [8]
      J2025-11-28T15:28:58.335400954Z INFO:     Waiting for application startup.
      G2025-11-28T15:28:58.335540687Z INFO:     Application startup complete.
      R2025-11-28T15:28:59.626909551Z INFO:     10.0.1.5:39278 - "GET / HTTP/1.0" 200 OK
      R2025-11-28T15:29:06.519078194Z INFO:     10.0.1.5:56198 - "GET / HTTP/1.0" 200 OK
      d2025-11-28T15:29:06.695898424Z INFO:     10.0.1.5:56214 - "GET /favicon.ico HTTP/1.0" 404 Not Found



In [ ]:
#| export
def delete_app(
    server_url:str,  # CapRover server URL
    token:str,  # Authentication token from caplogin
    app_name:str  # Name of the app to delete
) -> dict:  # API response
    "Delete an app from CapRover"
    headers = {"x-captain-auth": token}
    body = {"appName": app_name}
    response = httpx.post(f"{server_url}api/v2/user/apps/appDefinitions/delete", json=body, headers=headers)
    if response.status_code != 200: raise Exception(f'HTTP Error: {response}')
    if response.json()['status'] != 100: raise Exception(f'API Error: {response.json()}')
    return response.json()

In [ ]:
delete_app(capsrv, token, app_name)

{'status': 100, 'description': 'App is deleted', 'data': {}}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()